# Data Generation (Supervised Learning)

## Import Packages

In [2]:
import numpy as np
import scipy
from scipy.stats import norm
import pandapower as pp
import pandas as pd
import pandapower.networks
import tensorflow as tf
import csv
import torch

## Input Data

In [35]:
case = pp.networks.case30()
number_of_bus = case.bus.shape[0]
S_base = 100

### Z_base

V_base = np.zeros((case.line.shape[0],1))
for  i in range(case.line.shape[0]):
    V_base[i,0] = case.bus.loc[case.bus.index[case.bus.loc[:,'name']==case.line.loc[i,'to_bus']],'vn_kv']

Z_base = np.square(V_base)/S_base

### P_Gen

P_Gen = np.zeros((case.line.shape[0],1))
for i in range(case.gen.shape[0]):
    indices = case.line.index[case.line.loc[:,'from_bus']==case.gen.loc[i,'bus']]  
    P_Gen[indices,0] = case.gen.loc[i,'p_mw']/S_base
           
### R X  PL QL PG initial_voltage initial_delta
R = np.multiply(np.array([case.line.loc[:,'r_ohm_per_km']]),np.array([case.line.loc[:,'length_km']])).transpose()
X = np.multiply(np.array([case.line.loc[:,'x_ohm_per_km']]),np.array([case.line.loc[:,'length_km']])).transpose()

Data_form = np.hstack((np.multiply(R,1/Z_base),\
                       (X/Z_base)))

In [63]:
np.array([case.line.loc[:,'r_ohm_per_km']]) * np.array([case.line.loc[:,'length_km']]).transpose()

SyntaxError: invalid syntax (Temp/ipykernel_12652/3868878392.py, line 1)

In [67]:
1/Z_base

array([[0.00548697],
       [0.00548697],
       [0.00548697],
       [0.00548697],
       [0.00548697],
       [0.00548697],
       [0.00548697],
       [0.00548697],
       [0.00548697],
       [0.00548697],
       [0.00548697],
       [0.00548697],
       [0.00548697],
       [0.00548697],
       [0.00548697],
       [0.00548697],
       [0.00548697],
       [0.00548697],
       [0.00548697],
       [0.00548697],
       [0.00548697],
       [0.00548697],
       [0.00548697],
       [0.00548697],
       [0.00548697],
       [0.00548697],
       [0.00548697],
       [0.00548697],
       [0.00548697],
       [0.00548697],
       [0.00548697],
       [0.00548697],
       [0.00548697],
       [0.00548697],
       [0.00548697],
       [0.00548697],
       [0.00548697],
       [0.00548697],
       [0.00548697],
       [0.00548697],
       [0.00548697]])

## Roulette Wheel Algorithm Using Normal-PDF to Generate Scenarios

In [8]:
number_of_data = 10000       # Including data for both Training and Testing

Scenario = np.zeros([number_of_data,Data_form.shape[0],Data_form.shape[1]])

for i in range(number_of_data):
    
    Scenario[i,:,:] = Data_form
    Scenario[i,load_bus_number,2:4] = norm.ppf(np.random.rand(load.shape[0],1),\
                                                              load,np.multiply(0.1,load))

Training_Data = Scenario[0:number_of_training_data,:,:]
Test_Data = Scenario[number_of_training_data:,:,:]

## Implement Power Flow to Get Labels

In [9]:
Label = np.zeros((number_of_data,number_of_bus,2))

for i in range(number_of_data):
    
    case.load.iloc[:,6:8] = Scenario[i,case.load.iloc[:,0],2:4]
    
    pp.runpp(case, algorithm='nr', calculate_voltage_angles='auto', init='auto',\
                  max_iteration='auto', tolerance_mva=1e-08, trafo_model='t',\
                  trafo_loading='current', enforce_q_lims=False,\
                  check_connectivity=True, voltage_depend_loads=True,\
                 consider_line_temperature=False)
    
    Label[i,:,:] = case.res_bus.iloc[:,0:2]
    
Training_Label = Label[0:number_of_training_data,:,:]
Test_Label = Label[number_of_training_data:,:,:]


## Exporting Data to CSV files

In [10]:
Final_Training_Data = pd.DataFrame(Training_Data.reshape(number_of_training_data,-1))
Final_Training_Label  = pd.DataFrame(Training_Label.reshape(number_of_training_data,-1))
Final_Training_Data.to_csv("C:/Users/adria/OneDrive/Desktop/DataGeneration/train_data_30bus.csv",index = False)
Final_Training_Label.to_csv("C:/Users/adria/OneDrive/Desktop/DataGeneration/train_label_30bus.csv",index = False)

Final_Test_Data   = pd.DataFrame(Test_Data.reshape(number_of_test_data,-1))
Final_Test_Label  = pd.DataFrame(Test_Label.reshape(number_of_test_data,-1))
Final_Test_Data.to_csv("C:/Users/adria/OneDrive/Desktop/DataGeneration/test_data_30bus.csv",index = False)
Final_Test_Label.to_csv("C:/Users/adria/OneDrive/Desktop/DataGeneration/test_label_30bus.csv",index = False)


